<a href="https://colab.research.google.com/github/N-django/data-analysis-vehicle-Insuranceclaims/blob/main/Premium_Pricing_Model(CF_%26_CS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Importing Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
df= pd.read_excel('/content/drive/MyDrive/Motor Private Dataset.xlsx')
df.columns

Index(['CMB_CLAIM_NO', 'POLICY_NO', 'EF', 'ET', 'LOSS_DATE', 'REPORT DATE',
       'Vehicle_ID', 'YOM', 'AGE_CATEGORY', 'CAR_AGE', ' COLORS',
       'CUBIC_CAPACITY', 'CARRY_CAPACITY', 'BODY TYPE', 'REG_Name',
       'VEHICLE MAKE', 'VEHICLE MODELS', 'GROSS_PAID'],
      dtype='object')

### Data Preprocessing

In [6]:
df.head()
blank_columns=df.isna().sum()
blank_columns

,0
CMB_CLAIM_NO,0
POLICY_NO,0
EF,0
ET,0
LOSS_DATE,0
REPORT DATE,0
Vehicle_ID,0
YOM,5439
AGE_CATEGORY,0
CAR_AGE,5439


Imputation of Missing Values

In [7]:
median_yom = df['YOM'].median()
df['YOM']= df['YOM'].fillna(median_yom)

mean_car_age = df['CAR_AGE'].mean()
mean_cubic_capacity = df['CUBIC_CAPACITY'].mean()
mean_carry_capacity = df['CARRY_CAPACITY'].mean()

df['CAR_AGE'] = df['CAR_AGE'].fillna(mean_car_age)
df['CUBIC_CAPACITY'] = df['CUBIC_CAPACITY'].fillna(mean_cubic_capacity)
df['CARRY_CAPACITY'] = df['CARRY_CAPACITY'].fillna(mean_carry_capacity)
df['REG_Name']=df['REG_Name'].fillna('UNKNOWN')
df['GROSS_PAID']=df['GROSS_PAID'].fillna(0)

Blank_columns = df.isna().sum()

df['Claim_Frequency'] = df.groupby('Vehicle_ID')['CMB_CLAIM_NO'].transform('count')
print(f"Total number of missing values: {Blank_columns}")

Total number of missing values: CMB_CLAIM_NO      0
POLICY_NO         0
EF                0
ET                0
LOSS_DATE         0
REPORT DATE       0
Vehicle_ID        0
YOM               0
AGE_CATEGORY      0
CAR_AGE           0
 COLORS           0
CUBIC_CAPACITY    0
CARRY_CAPACITY    0
BODY TYPE         0
REG_Name          0
VEHICLE MAKE      0
VEHICLE MODELS    0
GROSS_PAID        0
dtype: int64


Calculating Claim Frequency

In [8]:
df['Policy Years'] = df['EF'].str.extract(r'(\d{4})').astype(float)

# Group by Vehicle_ID and count the number of unique Policy Years
policy_period = df.groupby('Vehicle_ID')['Policy Years'].nunique().reset_index()
policy_period.rename(columns={'Policy Years': 'Policy Period'}, inplace=True)

# Merge back to the original dataframe
df = df.merge(policy_period, on='Vehicle_ID', how='left')

# Calculate Claim Frequency (CF)
df['CF'] = df['Claim_Frequency'] / df['Policy Period']
df

# # Export the modified dataframe to an Excel file
# output_file = '/content/Processed_Policies.xlsx'
# df.to_excel(output_file, index=False)

# # Provide a download link for the Excel file
# from google.colab import files
# files.download(output_file)

,CMB_CLAIM_NO,POLICY_NO,EF,ET,LOSS_DATE,REPORT DATE,Vehicle_ID,YOM,AGE_CATEGORY,CAR_AGE,...,CARRY_CAPACITY,BODY TYPE,REG_Name,VEHICLE MAKE,VEHICLE MODELS,GROSS_PAID,Claim_Frequency,Policy Years,Policy Period,CF
0,C/10/700/043075,10/700/0030439,06/09/2023,06/08/2024,2024-05-13,2024-05-14 00:00:00,KCY 676R,2014.0,8+ Years,10.000000,...,5.000000,STATION WAGON,TOYOTA HARRIER HYBDRID,TOYOTA,TOYOTA HARRIER,0.0,1,2023.0,1,1.0
1,C/12/700/041177,12/700/0071745,03/01/2023,02/29/2024,2024-01-26,2024-02-15 00:00:00,KCM 432W,2011.0,8+ Years,13.000000,...,5.000000,STATION WAGON,TOYOTA RAV4,TOYOTA,TOYOTA RAV4,0.0,1,2023.0,1,1.0
2,C/40/700/037888,AE700/0065804/000/00,03/03/2023,01/24/2024,2023-08-30,2023-09-01 00:00:00,KDL 192F,2017.0,4-7 Years,7.000000,...,5.000000,STATION WAGON,SUBARU FORESTER,SUBARU,SUBARU FORESTER,0.0,1,2023.0,1,1.0
3,C/10/700/036490,10/700/0101357,01/01/2023,12/31/2023,2023-04-14,2023-06-19 00:00:00,KBZ 107K,2007.0,8+ Years,17.000000,...,5.000000,SEDAN,SALOON AUTO,UNKNOWN,SALOON AUTO,0.0,2,2023.0,2,1.0
4,C/15/700/033788,AI700/0060894/000/00,09/21/2022,07/20/2023,2023-01-23,2023-01-30 00:00:00,KCH 319U,2009.0,8+ Years,15.000000,...,5.000000,STATION WAGON,NISSAN XTRAIL,NISSAN,NISSAN X-TRAIL,0.0,3,2022.0,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42432,10/070/2005/1610,P/10/2005/0710/3668,05/08/2005,12/31/2005,2005-11-30,2005-11-30 00:00:00,KAJ 774B,2004.0,8+ Years,27.000000,...,5.000000,SEDAN,MITSUBISHI LANCER,MITSUBISHI,MITSUBISHI LANCER,0.0,1,2005.0,1,1.0
42433,ZAA700-/002188,ZAA700/019105/000/00,01/01/2005,12/31/2005,2005-10-23,2005-10-23 00:00:00,KAE 042B,2004.0,8+ Years,29.000000,...,5.000000,STATION WAGON,PEUGEOT 405 GR,GEO,PEUGEOT 405,0.0,2,2005.0,1,2.0
42434,ZAA700/003507,ZAA700/020286/000/00,11/09/2004,11/08/2005,2005-07-05,2005-07-07 10:36:00,KAR 387T AA,2004.0,8+ Years,29.000000,...,5.000000,STATION WAGON,TOYOTA,TOYOTA,TOYOTA,0.0,1,2004.0,1,1.0
42435,10/070/2005/1247,P/10/2005/0710/3605,04/27/2005,04/26/2006,2005-06-02,2005-06-02 00:00:00,KAD 261S,2004.0,8+ Years,36.000000,...,5.000000,SEDAN,NISSAN,NISSAN,NISSAN,0.0,1,2005.0,1,1.0


Get Claim Severity

In [10]:
df_grouped = df.groupby('Vehicle_ID').agg({'GROSS_PAID': 'sum'}).reset_index()
df_grouped.rename(columns={'GROSS_PAID': 'Total_Claim_Amount'}, inplace=True)
df = pd.merge(df, df_grouped, on='Vehicle_ID', how='left')
df['Severity']=df['Total_Claim_Amount']/df['Claim_Frequency']
df.columns


Index(['CMB_CLAIM_NO', 'POLICY_NO', 'EF', 'ET', 'LOSS_DATE', 'REPORT DATE',
       'Vehicle_ID', 'YOM', 'AGE_CATEGORY', 'CAR_AGE', ' COLORS',
       'CUBIC_CAPACITY', 'CARRY_CAPACITY', 'BODY TYPE', 'REG_Name',
       'VEHICLE MAKE', 'VEHICLE MODELS', 'GROSS_PAID', 'Claim_Frequency',
       'Policy Years', 'Policy Period', 'CF', 'Total_Claim_Amount',
       'Severity'],
      dtype='object')

Variance of CF and CS

In [16]:
print("Mean Claim Frequency",df['CF'].mean())
print( "Variance Claim Frequency",df['CF'].var())

print("&")

print("Mean Severity",df['Severity'].mean())
print( "Variance Severity",df['Severity'].var())

Mean Claim Frequency 1.20495530068662
Variance Claim Frequency 0.178888507230115
&
Mean Severity 199704.0339148088
Variance Severity 152412596022.94797


In [18]:
selected_columns=['Vehicle_ID', 'CAR_AGE', ' COLORS','CUBIC_CAPACITY', 'CARRY_CAPACITY', 'BODY TYPE','VEHICLE MAKE', 'VEHICLE MODELS', 'Claim_Frequency', 'CF','Severity']
df=selected_columns

Correlation Analysis